In [112]:
def complementaire(I, n):
    J= []
    for i in range(1,n+1):
        if i not in I:
            J.append(i)
    return set(J)


def pds(s):
    p = 0
    for i in s:
        if i != 0:
            p += 1
    return p


n = 7
k = 4
S = Subsets(n,k)

H = Matrix(GF(2),[[1,1,0,1,1,0,0],[1,0,1,1,0,1,0],[0,1,1,1,0,0,1]])
s = [1,0,0,0,1,1,0]
w = 5

#Entrées : 
#      - matrice de parité H
#      - vecteur syndrome s
#      - w dans [0,n]
#Sortie : 
#      - vecteur d'erreur e tel que He^T=s et |e|<w

def Prange(H, s, w):
    p = w + 1
    while p > w:
        I = S.random_element()
        J = complementaire(I, n)
        #print(I)
        #print(J)
        Hj = H[[k for k in range(0,n-k)],[k for k in range(0,n) if (k+1) in J ]]
        if Hj.determinant() == 0:
            continue
        sj = vector([s[k] for k in range(0,n) if (k+1) in J])
        Hinv = Hj.inverse()
        sp = Hinv*sj
        #print(sp)
        #print(pds(sp))
        p = pds(sp)
    j = 0
    e = []
    for i in range(n):
        if i in J:
            e.append(sp[j])
            j+= 1
        else:
            e.append(0)
    return e;

In [115]:
Prange(H, s, w)

[0, 0, 1, 0, 0, 0, 0]